# Semantic segmentation test

---

**Author:** [rodoart](https://github.com/rodoart/)<br>
**Date created:** 2021/07/24<br>
**Last modified:** 2021/07/24<br>
**Description:** 
Using pretrained neural networks to segment images of rooms.

# Semantic segmentation of rooms

## Path config

If you want the files to be copied to another folder within the same machine you are working on, by a source path other than remote.

In [1]:
PROJECT_SLUG = 'vigilancia_mascotas'
NAME = 'semantic_segmentation_test_with_room_data'
NUMBER = '0.1'

NOTEBOOK_NAME = f'{NUMBER}-{PROJECT_SLUG}-{NAME}.ipynb'

# COLAB
DRIVE_MOUNT = '/drive'
REMOTE_PATH = f'{DRIVE_MOUNT}/MyDrive/IA/seminario_innovacion/{PROJECT_SLUG}'
LOCAL_PATH = '.'
NOTEBOOK_PATH = f'{DRIVE_MOUNT}/MyDrive/Colab Notebooks/{NOTEBOOK_NAME}'

# LOCAL
#REMOTE_PATH = '..'
#LOCAL_PATH = '..'
#DRIVE_MOUNT = ''
#NOTEBOOK_PATH = f'G:\\Mi unidad\\Colab Notebooks\\{NOTEBOOK_NAME}'

In [2]:
if DRIVE_MOUNT:
    from google.colab import drive
    drive.mount(DRIVE_MOUNT)
    to_remote = True
else:
    to_remote = False

Mounted at /drive


## Path functions

In [3]:
import sys

# It depends on where the library that comes with this package is stored.
sys.path.append(REMOTE_PATH)

In [4]:
from vigilancia_mascotas.utils.paths \
    import make_remote_copy_of_workspace_functions

In [5]:
local_dir, update_from_remote, update_to_remote, update_notebook = \
    make_remote_copy_of_workspace_functions(
        local_path=LOCAL_PATH,
        remote_path=REMOTE_PATH,
        notebook_path = NOTEBOOK_PATH
    )

In [6]:
from pathlib import Path

## Update local folders

In [7]:
#update_from_remote()

## Prepare the dataset

### Download

For the development of this experiment, the data set of Unity Computer Vision Datasets (Home interior sample) is used, which contains 1,000 artificially created with different types of labeling, including semantic segmentation.

![Example from the dataset](https://content.cdntwrk.com/files/aHViPTEwODAwMiZjbWQ9aXRlbWVkaXRvcmltYWdlJmZpbGVuYW1lPWl0ZW1lZGl0b3JpbWFnZV82MGY5YmY3MGYxNmZkLnBuZyZ2ZXJzaW9uPTAwMDAmc2lnPTIzZTcwMzIwNzIzMGIyMzAwNmZkM2VhNGZiODFiYzkz)

Access to the dataset can be obtained directly on the provided page. The following download section is designed to work only for contributors to this project, but can be easily modified once you get the download link.

In [8]:
import requests

In [9]:
file_name = "Unity_Residential_Interiors.zip"

url_dataset_zip = "https://storage.googleapis.com/computer-vision-datasets/"\
                + f"residential/{file_name}"

zip_destiny_path = local_dir('tmp', file_name)

In [10]:
if not zip_destiny_path.is_file():
    zip_destiny_path.parent.mkdir(exist_ok=True)
    response = requests.get(url_dataset_zip)
    open(zip_destiny_path, "wb").write(response.content)

### Unzip the files

In [11]:
import zipfile
from vigilancia_mascotas.utils.paths import is_valid

In [12]:
unzip_path = local_dir('tmp', file_name[:-4].lower())

In [13]:
if not is_valid(unzip_path):
    with zipfile.ZipFile(zip_destiny_path, 'r') as zip_ref:
        zip_ref.extractall(unzip_path)


### Moving files

In [14]:
import shutil
from os import makedirs

In [15]:
dataset_path = local_dir('data','raw','semantic_segmentation', 'unity_residential_interiors')

makedirs(dataset_path, exist_ok=True)

In [16]:
tmp_images_dir = unzip_path.joinpath(
    'Residential Interiors Dataset',
    'RGB2d9da855-d495-4bd9-9a1f-6744db5a3249')

raw_images_dir = dataset_path.joinpath('images')

tmp_labels_dir = tmp_images_dir.parent.joinpath(
    'SemanticSegmentation50529ca2-588c-4aba-ab92-a9d4a31a56d4'
)
raw_labels_dir = dataset_path.joinpath('labels')

In [17]:
for source, destiny in zip([tmp_images_dir, tmp_labels_dir], 
                           [raw_images_dir, raw_labels_dir]):
    if not is_valid(destiny):
        shutil.move(
            src=source,
            dst=destiny
        )

### Separate train and test subsets

Shuffle file names

In [18]:
from os import listdir

In [19]:
# Obtain file_names
file_names = [name[4:-4] for name in listdir(raw_images_dir) if name.endswith('.png')]
len(file_names)

1000

In [20]:
from random import shuffle

In [21]:
shuffle(file_names)

In [22]:
n_train = round(len(file_names)*0.7)

train_names = file_names[:n_train]
val_names = file_names[n_train:]

### Move files to train and test folders

In [23]:
dataset_processed_path = local_dir('data','processed','semantic_segmentation', 'unity_residential_interiors')

In [24]:
from os import mkdir
from os import rmdir

In [25]:
if not is_valid(dataset_processed_path):

    dataset_processed_path.mkdir(parents=True, exist_ok=True)

    for set_name, file_names in zip(['train', 'val'], [train_names, val_names]):
        for set_type, name_prefix in zip(['images', 'labels'], 
                                        ['rgb', 'segmentation']):
            for file_name in file_names:
                destiny_path = dataset_processed_path.joinpath(
                    f'{set_name}_{set_type}',
                    f'{file_name}.png'
                )

                origin_path = dataset_path.joinpath(
                    set_type, f'{name_prefix}_{file_name}.png'
                )

                if not destiny_path.parent.is_dir():
                    destiny_path.parent.mkdir(exist_ok=True)

                shutil.copy(origin_path, destiny_path)

## Pre-trained models selection

In [26]:
!pip install keras_segmentation
!pip install keras==2.4.3
!pip install tensorflow==2.4.1


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.3 MB 11.6 MB/s 
  Created wheel for keras-segmentation: filename=keras_segmentation-0.3.0-py3-none-any.whl size=29071 sha256=a97ac8fad73637aded4116717b287fa4058f1990ffe34dac33985b2b23ed1d35
  Stored in directory: /root/.cache/pip/wheels/68/dc/b3/596a3e2461ba16e935ef31661c26e823f841cfb577cec4c47a
Successfully built keras-segmentation
  Attempting uninstall: imageio
    Found existing installation: imageio 2.4.1
    Uninstalling imageio-2.4.1:
      Successfully uninstalled imageio-2.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/

In [27]:
from keras_segmentation.pretrained import pspnet_50_ADE_20K , pspnet_101_cityscapes, pspnet_101_voc12


In [28]:
val_images_dir = dataset_processed_path.joinpath('val_images')
val_labels_dir = dataset_processed_path.joinpath('val_labels')

In [29]:
all_evaluations = {}

for model_function in [pspnet_50_ADE_20K, pspnet_101_cityscapes, pspnet_101_voc12]:
  model = model_function()
  eval_dict = model.evaluate_segmentation(inp_images_dir=val_images_dir, 
                              annotations_dir=val_labels_dir )
  
  all_evaluations[model_function.__name__] = eval_dict 


187842560/187839752 [==============================] - 3s 0us/step


300it [40:16,  8.06s/it]


264151040/264145248 [==============================] - 3s 0us/step


300it [1:32:54, 18.58s/it]


264151040/264149344 [==============================] - 3s 0us/step


300it [40:54,  8.18s/it]


In [54]:
for key, value in all_evaluations.items():
  all_evaluations[key]['class_wise_IU'] = value['class_wise_IU'].tolist()

AttributeError: ignored

In [64]:
for key, value in all_evaluations.items():
  print(key)
  for in_key, in_value in value.items():
    if in_key != 'class_wise_IU':
      print(f'{in_key}: {in_value}')
  print('')

pspnet_50_ADE_20K
frequency_weighted_IU: 0.48376047458080507
mean_IU: 0.003779825647524711

pspnet_101_cityscapes
frequency_weighted_IU: 0.19482523516014133
mean_IU: 0.010858835360365951

pspnet_101_voc12
frequency_weighted_IU: 0.8811641102018097
mean_IU: 0.04279866287125835



In [ ]:
ppsnet 101 

https://arxiv.org/pdf/1411.4038.pdf

In [56]:
import json 
remote_path = Path(REMOTE_PATH).resolve()

In [57]:
with open(remote_path.joinpath('data', 'interim', 'first_results.json'), 'w') as fp:
    json.dump(all_evaluations, fp)

## Update remote folders

In [ ]:
update_notebook('to_remote')

In [ ]:
update_to_remote()